In [1]:
import requests
import xmltodict
import json # json 처리
import pymysql
import time
import random
from datetime import datetime

In [3]:
# !chcp 65001

Active code page: 65001


In [3]:
# !conda info --envs 환경

# conda environments:
#
base                     C:\ProgramData\Miniconda3
ssac-one              *  C:\ProgramData\Miniconda3\envs\ssac-one
                         C:\Users\kjk66\anaconda3



# 코로나 해외 발병현황(20200101 - 20201231)

In [2]:
# 코로나 OpenAPI 연계
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson'
api_key = '3fFyL9wc7wsGOpUkhxQchp0AZBH8PlyNELGTdLSBGeXMNgaPRcHbWZ%2FsKlNnnw4q67omLLYJM34Yu1osH58d9w%3D%3D'

qs = 'ServiceKey={0}&startCreateDt={1}&endCreateDt={2}'.format(api_key, '20200101', '20201231') 

response = requests.get(url + "?" + qs)
response_dict = xmltodict.parse(response.content) # xml -> ordereddict
response_json = json.dumps(response_dict, ensure_ascii=False) # ordereddict -> json-string
response_dict2 = json.loads(response_json) # json-stirng -> general dict
values = response_dict2['response']['body']['items']['item']

In [4]:
values[0]

{'areaNm': '기타',
 'areaNmCn': '其他',
 'areaNmEn': 'Others',
 'createDt': '2020-12-31 10:14:39.677',
 'natDeathCnt': '3785',
 'natDeathRate': '1.1921109655',
 'natDefCnt': '317504',
 'nationNm': '기타지역 영토',
 'nationNmCn': '其他',
 'nationNmEn': 'Others',
 'seq': '67239',
 'stdDay': '2020년 12월 31일 09시',
 'updateDt': 'null'}

In [4]:
# 코로나 해외발생현황 테이블 생성 (mysql 작업)
"""
create table global_def_cnt(
	area varchar(30), 
    area_cn varchar(30),
    area_en varchar(30), 
    creat_dt datetime not null, 
    nat_death_cnt int, 
    nat_death_rate double, 
    nat_def_cnt int, 
    nat varchar(30) not null, 
    nat_cn varchar(30), 
    nat_en varchar(100), 
    seq int primary key, 
    stdday date not null, 
    update_dt datetime);  
"""

# 응답메시지 명세
"""
global_covid_desc = {'areaNm' : '지역명', 
                     'areaNmCn' : '지역명(중문)', 
                     'areaNmEn' : '지역명(영문)', 
                     'createDt' : '등록일시분초', 
                     'natDeathCnt' : '국가별 사망자 수', 
                     'natDeathRate' : '확진률 대비 사망률', 
                     'natDefCnt' : '국가별 확진자 수', 
                     'nationNm' : '국가명', 
                     'nationNmCn' : '국가명(중문)', 
                     'nationNmEn' : '지역명(영문)', 
                     'seq' : '게시글번호(고유)', 
                     'stdDay' : '기준일시', 
                     'updateDt' : '수정일시분초'}
"""

'\ncreate table global_def_cnt(\n\tarea varchar(30), \n    area_cn varchar(30),\n    area_en varchar(30), \n    creat_dt datetime not null, \n    nat_death_cnt int, \n    nat_death_rate double, \n    nat_def_cnt int, \n    nat varchar(30) not null, \n    nat_cn varchar(30), \n    nat_en varchar(30), \n    seq int primary key, \n    stdday date not null, \n    update_dt datetime);  \n'

In [6]:
# api 파라미터별 컬럼 매핑
columns = ['area', 'area_cn', 'area_en', 'creat_dt', 'nat_death_cnt', 'nat_death_rate', 'nat_def_cnt', 'nat', 'nat_cn', 'nat_en', 'seq', 'stdday', 'update_dt']

keys = ['areaNm', 'areaNmCn', 'areaNmEn', 'createDt', 'natDeathCnt', 'natDeathRate', 'natDefCnt', 'nationNm',	'nationNmCn', 'nationNmEn', 'seq', 'stdDay', 'updateDt']

key_column_map = { key: column for key, column in zip(keys, columns) }
print( key_column_map )
column_key_map = { column: key for key, column in zip(keys, columns) }
print( column_key_map )

len(keys), len(columns)

{'areaNm': 'area', 'areaNmCn': 'area_cn', 'areaNmEn': 'area_en', 'createDt': 'creat_dt', 'natDeathCnt': 'nat_death_cnt', 'natDeathRate': 'nat_death_rate', 'natDefCnt': 'nat_def_cnt', 'nationNm': 'nat', 'nationNmCn': 'nat_cn', 'nationNmEn': 'nat_en', 'seq': 'seq', 'stdDay': 'stdday', 'updateDt': 'update_dt'}
{'area': 'areaNm', 'area_cn': 'areaNmCn', 'area_en': 'areaNmEn', 'creat_dt': 'createDt', 'nat_death_cnt': 'natDeathCnt', 'nat_death_rate': 'natDeathRate', 'nat_def_cnt': 'natDefCnt', 'nat': 'nationNm', 'nat_cn': 'nationNmCn', 'nat_en': 'nationNmEn', 'seq': 'seq', 'stdday': 'stdDay', 'update_dt': 'updateDt'}


(13, 13)

In [ ]:
connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'Pa$$w0rd', 'db': 'prj1', 'charset': 'utf8' }
conn = pymysql.connect(**connection_info)
cursor = conn.cursor()

for value in values:

    dt = datetime.strptime(value['createDt'], "%Y-%m-%d %H:%M:%S.%f")
    value['createDt'] = datetime.strftime(dt, '%Y-%m-%d %H:%M:%S')
    dt = datetime.strptime(value['stdDay'], "%Y년 %m월 %d일 %H시")
    value['stdDay'] = datetime.strftime(dt, '%Y-%m-%d')


    try: value['areaNmCn'] = value['areaNmCn'] if value['areaNmCn'] else '-1' 
    except: value['areaNmCn'] = '-1'

    try: value['areaNmEn'] = value['areaNmEn'] if value['areaNmEn'] else '-1'
    except: value['areaNmEn'] = '-1'

    try: value['nationNmCn'] = value['nationNmCn'] if value['nationNmCn'] else '-1'
    except: value['nationNmCn'] = '-1'

    try: value['nationNmEn'] = value['nationNmEn'] if value['nationNmEn'] else '-1'
    except: value['nationNmEn'] = '-1'  

    if not value['updateDt'] or value['updateDt'] == 'null':
        value['updateDt'] = value['createDt']
    else:
        dt = datetime.strptime(value['updateDt'], "%Y-%m-%d %H:%M:%S.%f")
        value['updateDt'] = datetime.strftime(dt, '%Y-%m-%d %H:%M:%S')

    # value['qurRate'] = value['qurRate'] if value['qurRate'].isdigit() else '-1'  

    sql = 'INSERT INTO global_def_cnt VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
    cursor.execute(sql, [ value[column_key_map[column]] for column in columns ])


conn.commit()

conn.close()

# 미세먼지 데이터

In [6]:
# https://data.seoul.go.kr/dataList/OA-2218/S/1/datasetView.do
# 서울시 일별 평균 대기오염도 정보 (2019~2020)
# 파일데이터(csv)

In [26]:
import csv
import re

dust_info = []
with open('D:\ssac-one\demo-repo\data\일별평균대기오염도.csv') as f:
    reader = csv.reader(f)
    header = next(reader)
    for line in reader:        
        for i in range(3, 9):
            line[i] = -1 if line[i] == '' else line[i]
        dust_info.append(line)

In [3]:
# 미세먼지 관측정보 테이블 생성 (mysql 작업)
"""
create table air_quality(
    nation varchar(20) not null,
	obs_dt date not null,
    stn_nm varchar(20),
    no2 double default -1,
    o3 double default -1,
    co2 double default -1,
    so2 double default -1,
    pm10 int default -1,
    pm25 int default -1);
"""

'\ncreate table obs_dust(\n\tobs_dt date not null,\n    stn_nm varchar(20) primary key,\n    no2 double,\n    o3 double,\n    co2 double,\n    so2 double,\n    pm10 int,\n    pm25 int);\n'

In [27]:
dust_info[8]

['독일', '2020-10-09', '베를린', '18', '20', '-1', '-1', '12', '-1']

In [29]:
connection_info = { 'host': 'localhost', 'user': 'root', 'password': 'Pa$$w0rd', 'db': 'prj1', 'charset': 'utf8' }
conn = pymysql.connect(**connection_info)
cursor = conn.cursor()


sql = 'INSERT INTO air_quality VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)'
cursor.executemany(sql, dust_info) 
conn.commit()

conn.close()

In [7]:
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2019, 1, 1)
end_date = date(2021, 1, 1)
data_range = []
for single_date in daterange(start_date, end_date):
    data_range.append(single_date.strftime("%Y%m%d"))
data_range

['20190101',
 '20190102',
 '20190103',
 '20190104',
 '20190105',
 '20190106',
 '20190107',
 '20190108',
 '20190109',
 '20190110',
 '20190111',
 '20190112',
 '20190113',
 '20190114',
 '20190115',
 '20190116',
 '20190117',
 '20190118',
 '20190119',
 '20190120',
 '20190121',
 '20190122',
 '20190123',
 '20190124',
 '20190125',
 '20190126',
 '20190127',
 '20190128',
 '20190129',
 '20190130',
 '20190131',
 '20190201',
 '20190202',
 '20190203',
 '20190204',
 '20190205',
 '20190206',
 '20190207',
 '20190208',
 '20190209',
 '20190210',
 '20190211',
 '20190212',
 '20190213',
 '20190214',
 '20190215',
 '20190216',
 '20190217',
 '20190218',
 '20190219',
 '20190220',
 '20190221',
 '20190222',
 '20190223',
 '20190224',
 '20190225',
 '20190226',
 '20190227',
 '20190228',
 '20190301',
 '20190302',
 '20190303',
 '20190304',
 '20190305',
 '20190306',
 '20190307',
 '20190308',
 '20190309',
 '20190310',
 '20190311',
 '20190312',
 '20190313',
 '20190314',
 '20190315',
 '20190316',
 '20190317',
 '20190318',